# Explicación del Algoritmo NavGlobalPlanner

Este notebook explica y simula en Python el funcionamiento del `NavGlobalPlanner` (originalmente en C++). 
El objetivo es entender cómo procesa el archivo `global_plan.csv` para construir un grafo de navegación y planificar rutas.

## Pasos Principales:
1.  **Carga y Parseo**: Leer `global_plan.csv` e interpretar los diferentes tipos de datos (nodos del grafo, obstáculos, pozos).
2.  **Construcción del Grafo**: Crear un grafo dirigido donde los nodos son puntos en el espacio (x, y, orientación) y las aristas son las conexiones posibles.
3.  **Búsqueda de Rutas (A*)**: Implementar el algoritmo A* con la misma función de costo y heurística que usa el código C++.
4.  **Visualización**: Mostrar el mapa, el grafo y la ruta resultante.

## Clarificación de Roles: NavGlobalPlanner vs move_base Global Planner

Es importante distinguir entre `NavGlobalPlanner` y el planificador global utilizado por `move_base`.

1.  **NavGlobalPlanner (Mission Planner):**
    *   Este es el componente que estamos analizando en este notebook.
    *   Actúa como un **Gestor de Misiones** o planificador de alto nivel.
    *   Trabaja sobre un grafo estático predefinido en `global_plan.csv`.
    *   Su objetivo es encontrar la secuencia de nodos (waypoints) para ir de un punto A a un punto B en la mina, respetando la topología del grafo.
    *   No tiene en cuenta el mapa de costos local ni obstáculos dinámicos en tiempo real (aunque publica obstáculos estáticos para que otros los vean).
    *   Envía el camino global (lista de waypoints) al `NavigationServer`.

2.  **move_base Global Planner (`global_planner/GlobalPlanner`):**
    *   Es el planificador global estándar de ROS que utiliza `move_base`.
    *   Recibe los objetivos (waypoints) del `NavigationServer` (que a su vez los recibió de `NavGlobalPlanner`).
    *   Planifica una ruta detallada y libre de obstáculos en el mapa de costos global (`global_costmap`) para llegar al siguiente waypoint.
    *   Trabaja en conjunto con el planificador local (TEB) para la evasión de obstáculos y el control del robot.

**Flujo de Navegación:**
`NavGlobalPlanner` (Ruta en Grafo) -> `NavigationServer` (Orquestador) -> `move_base` (Planificación detallada y Control)

In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import math
from shapely.geometry import Point, Polygon, LineString
import heapq

## 1. Carga y Parseo del CSV

Cargamos el archivo `global_plan.csv`. Este archivo contiene toda la información estática necesaria para la navegación.

In [2]:
import ipywidgets as widgets
import io
import pandas as pd

print("Selecciona el archivo global_plan.csv desde tu equipo:")
uploader = widgets.FileUpload(
    accept='.csv',  # Aceptar solo archivos .csv
    multiple=False  # Solo un archivo
)
display(uploader)

Selecciona el archivo global_plan.csv desde tu equipo:


FileUpload(value=(), accept='.csv', description='Upload')

In [3]:
# Cargar el archivo seleccionado
if 'uploader' in locals() and uploader.value:
    try:
        # Manejar diferentes versiones de ipywidgets
        uploaded_file = None
        if isinstance(uploader.value, (tuple, list)):
            # ipywidgets >= 8.0
            uploaded_file = uploader.value[0]
        elif isinstance(uploader.value, dict):
            # ipywidgets < 8.0
            key = list(uploader.value.keys())[0]
            uploaded_file = uploader.value[key]
            
        if uploaded_file:
            print(f"Procesando archivo: {uploaded_file['name']}")
            content = uploaded_file['content']
            # 'content' son bytes
            df = pd.read_csv(io.BytesIO(content))
            print("Archivo cargado correctamente.")
            print(df.head())
        else:
            raise ValueError("Formato de archivo subido no reconocido.")
            
    except Exception as e:
        print(f"Error al leer el archivo subido: {e}")
        # Fallback a dummy data o error
        df = None
else:
    print("No se ha seleccionado ningún archivo en el widget anterior.")
    print("Intentando cargar archivo por defecto...")
    csv_path = './test_data/4_explain_nav_global_planner/global_plan.csv'
    try:
        df = pd.read_csv(csv_path)
        print(f"Cargado desde ruta por defecto: {csv_path}")
        print(df.head())
    except FileNotFoundError:
        print(f"No se encontró el archivo por defecto en {csv_path}.")
        # Crear un dataframe dummy si no existe el archivo para demostración
        data = {
            'type': ['graph_pose', 'graph_pose', 'graph_pose', 'hole'],
            'pose_type': ['home_pose', 'street', 'street', 'hole'],
            'graph_id': [0, 1, 2, 3],
            'drillhole_id': [-1, -1, -1, 101],
            'graph_pose_local': ['0,0,0', '10,0,0', '20,0,0', '20,5,1.57'],
            'connections': ['1', '0,2', '1', ''],
            'local_geometry': ['', '', '', 'POINT (20 5)']
        }
        df = pd.DataFrame(data)
        print("Usando datos de ejemplo.")

No se ha seleccionado ningún archivo en el widget anterior.
Intentando cargar archivo por defecto...
Cargado desde ruta por defecto: ./test_data/4_explain_nav_global_planner/global_plan.csv
   Unnamed: 0     z  z_hole  drillhole_id  mesh      source  \
0           0  11.8    10.0          19.0  peld  hol_import   
1           1  11.2    10.0          18.0  peld  hol_import   
2           2  30.0    10.0          16.0  peld  hol_import   
3           3  25.0    10.0          14.0  peld  hol_import   
4           4  11.6    10.0          12.0  peld  hol_import   

                                     utm_coordinates  \
0  {'x': 343348.5100573936, 'y': 6334425.48609452...   
1  {'x': 343353.3511223035, 'y': 6334426.18903766...   
2  {'x': 343358.49319460144, 'y': 6334426.8051789...   
3  {'x': 343363.4353070089, 'y': 6334427.60238899...   
4  {'x': 343368.3796576862, 'y': 6334428.11002413...   

                                       geometry  unkown0  unkown1  ...  \
0   POINT (343348.51

## 2. Construcción del Grafo

Iteramos sobre el DataFrame para construir el grafo. 
- **Nodos**: Se crean a partir de `graph_pose_local` (x, y, theta).
- **Aristas**: Se crean a partir de la columna `connections`.

In [4]:
G = nx.DiGraph() # Grafo dirigido
pos = {}
node_data = {}
drillhole_to_node = {}

obstacles = []
holes = []

for index, row in df.iterrows():
    row_type = row['type']
    
    if row_type == 'graph_pose':
        # Parsear graph_pose_local "x,y,theta"
        try:
            pose_str = row['graph_pose_local'].replace('[', '').replace(']', '')
            parts = [float(x) for x in pose_str.split(',')]
            x, y, theta = parts[0], parts[1], parts[2]
            
            if pd.isna(row['graph_id']): continue
            node_id = int(row['graph_id'])
            
            # Agregar nodo al grafo
            G.add_node(node_id, x=x, y=y, theta=theta, type=row['pose_type'])
            pos[node_id] = (x, y)
            node_data[node_id] = {'x': x, 'y': y, 'theta': theta}
            
            # Map drillhole_id to node_id if valid
            if 'drillhole_id' in row and pd.notna(row['drillhole_id']) and row['drillhole_id'] != -1:
                drillhole_to_node[int(row['drillhole_id'])] = node_id
            
            # Parsear conexiones
            connections_str = str(row['connections']).replace('[', '').replace(']', '')
            if connections_str and connections_str != 'nan':
                conn_parts = connections_str.split(',')
                for conn in conn_parts:
                    if conn.strip():
                        target_id = int(conn)
                        # Las aristas se agregan después para asegurar que todos los nodos existan
                        # pero en networkx se pueden agregar igual.
                        # Sin embargo, necesitamos calcular el peso después.
                        pass
        except Exception as e:
            print(f"Error parsing row {index}: {e}")
            
    elif row_type == 'hole':
        # Parsear geometría del pozo para visualización
        # Formato esperado: POINT (x y)
        geo_str = str(row['local_geometry'])
        if 'POINT' in geo_str:
            coords = geo_str.replace('POINT (', '').replace(')', '').split()
            holes.append((float(coords[0]), float(coords[1]), row['drillhole_id']))

# Agregar aristas con pesos
# La función de peso en C++ es: dpos + dangle * angle_weight_ + 1.1
angle_weight = 1.0

def calculate_weight(n1, n2):
    dx = n1['x'] - n2['x']
    dy = n1['y'] - n2['y']
    dpos = math.sqrt(dx*dx + dy*dy)
    
    # Diferencia angular simplificada (en C++ usan cuaterniones)
    # Aquí asumimos theta en radianes
    # La lógica de C++ para dangle es: 2 * acos(dot_prod_quaternions)
    # Para 2D, podemos aproximar con la diferencia de ángulos
    
    # Replicando lógica de cuaterniones de C++:
    # q1 = (0, 0, sin(theta1/2), cos(theta1/2))
    # q2 = (0, 0, sin(theta2/2), cos(theta2/2))
    # dot = z1*z2 + w1*w2
    
    z1 = math.sin(n1['theta'] / 2)
    w1 = math.cos(n1['theta'] / 2)
    z2 = math.sin(n2['theta'] / 2)
    w2 = math.cos(n2['theta'] / 2)
    
    dot_prod = z1*z2 + w1*w2
    if dot_prod > 1.0: dot_prod = 0.9999
    if dot_prod < -1.0: dot_prod = -0.9999 # Safety
    
    dangle = 2 * math.acos(abs(dot_prod)) # abs porque q y -q son la misma rotación
    
    return dpos + dangle * angle_weight + 1.1

for index, row in df.iterrows():
    if row['type'] == 'graph_pose':
        source_id = int(row['graph_id'])
        connections_str = str(row['connections']).replace('[', '').replace(']', '')
        if connections_str and connections_str != 'nan':
            conn_parts = connections_str.split(',')
            for conn in conn_parts:
                if conn.strip():
                    target_id = int(conn)
                    if source_id in node_data and target_id in node_data:
                        weight = calculate_weight(node_data[source_id], node_data[target_id])
                        G.add_edge(source_id, target_id, weight=weight)

print("Pozos identificados (x, y, id):")
for h in holes:
    print(h)
print(f"Drillhole Mapping: {drillhole_to_node}")
print(f"Grafo construido con {G.number_of_nodes()} nodos y {G.number_of_edges()} aristas.")

Pozos identificados (x, y, id):
(49.48700845747953, 55.04271904844791, 19.0)
(54.32807336637052, 55.74566218815744, 18.0)
(59.47014566324651, 56.361803439445794, 16.0)
(64.41225807211595, 57.15901351161301, 14.0)
(69.35660874750465, 57.666648647747934, 12.0)
(74.26326038636034, 58.220500846393406, 10.0)
(53.54097737633856, 60.710145444609225, 17.0)
(58.858232379192486, 61.595875325612724, 15.0)
(64.08706290117698, 62.9614152899012, 13.0)
(68.89726796210743, 62.83744230866432, 11.0)
(73.93491642834852, 63.345367869362235, 9.0)
(75.2975526911323, 53.23252324573696, 8.0)
(67.75680594705045, 51.879938433878124, 6.0)
(61.460434952459764, 51.917291498743, 4.0)
(54.55055482912576, 51.093221958726645, 2.0)
(54.41641568264458, 43.49546741042286, 1.0)
(68.293032662943, 46.288253946229815, 5.0)
(76.26134032564005, 46.5451170867309, 7.0)
(61.98090187768685, 44.67360172048211, 3.0)
Drillhole Mapping: {19: 324, 18: 325, 16: 326, 14: 327, 12: 328, 10: 329, 17: 330, 15: 331, 13: 332, 11: 333, 9: 334, 

## 3. Algoritmo A*

Implementamos A* usando la heurística de distancia Euclidiana, tal como en `NavGlobalPlanner.cpp`.

In [5]:
def heuristic(u, v):
    n1 = node_data[u]
    n2 = node_data[v]
    return math.sqrt((n1['x'] - n2['x'])**2 + (n1['y'] - n2['y'])**2)

def run_astar(start_node, goal_node):
    try:
        path = nx.astar_path(G, start_node, goal_node, heuristic=heuristic, weight='weight')
        return path
    except nx.NetworkXNoPath:
        return None
    except Exception as e:
        print(f"Error en A*: {e}")
        return None


## 4. Visualización

Visualizamos el grafo completo, los pozos y la ruta encontrada.

In [6]:
import ipywidgets as widgets
from IPython.display import display

def visualize_path(start_node_id, goal_node_id):
    # Validar nodos
    if start_node_id not in G:
        print(f"Error: El nodo de inicio {start_node_id} no existe en el grafo.")
        return
    if goal_node_id not in G:
        print(f"Error: El nodo objetivo {goal_node_id} no existe en el grafo.")
        return

    # Ejecutar A*
    path = run_astar(start_node_id, goal_node_id)
    
    if path:
        print(f"Ruta encontrada: {path}")
    else:
        print("No se encontró ruta.")

    # Visualización
    plt.figure(figsize=(12, 12))
    
    # Dibujar obstáculos (si los hay)
    if holes:
        hx, hy, hids = zip(*holes)
        plt.scatter(hx, hy, c='red', marker='x', label='Pozos')
        for x, y, hid in holes:
            hid_str = str(int(hid)) if pd.notna(hid) else '?'
            plt.text(x, y, hid_str, fontsize=9, color='red')
        
    # Dibujar nodos del grafo
    gx, gy = zip(*pos.values())
    plt.scatter(gx, gy, s=5, c='gray', alpha=0.5, label='Nodos del Grafo')

    # Dibujar la ruta
    if path:
        path_x = [pos[n][0] for n in path]
        path_y = [pos[n][1] for n in path]
        plt.plot(path_x, path_y, '-b', linewidth=2, label='Ruta Global')
        plt.scatter(path_x, path_y, c='blue', s=20)
        
        # Inicio y Fin
        plt.scatter(pos[start_node_id][0], pos[start_node_id][1], c='green', s=100, marker='o', label='Inicio')
        plt.scatter(pos[goal_node_id][0], pos[goal_node_id][1], c='magenta', s=100, marker='*', label='Fin')

    plt.title(f"Planificación Global: {start_node_id} -> {goal_node_id}")
    plt.xlabel("X [m]")
    plt.ylabel("Y [m]")
    plt.legend()
    plt.axis('equal')
    plt.grid(True)
    plt.show()

# Widgets para selección de nodos
style = {'description_width': 'initial'}
drillhole_options = sorted(list(drillhole_to_node.keys())) if 'drillhole_to_node' in globals() else []
drillhole_selector = widgets.Dropdown(options=drillhole_options, description='Drillhole:', style=style)
start_node_widget = widgets.IntText(value=0, description='Nodo Inicio:', style=style)
goal_node_widget = widgets.IntText(value=347, description='Nodo Objetivo:', style=style)

# Contenedor de widgets

def on_drillhole_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        drillhole_id = change['new']
        if drillhole_id in drillhole_to_node:
            goal_node_widget.value = drillhole_to_node[drillhole_id]

drillhole_selector.observe(on_drillhole_change)

def update_view(start_node_id, goal_node_id):
    visualize_path(start_node_id, goal_node_id)

ui = widgets.HBox([start_node_widget, goal_node_widget, drillhole_selector])
out = widgets.interactive_output(update_view, {'start_node_id': start_node_widget, 'goal_node_id': goal_node_widget})
display(ui, out)


Output()